In [3]:
import os

from Crypto.Cipher import ChaCha20
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

def passwd():
    backend = default_backend()
    salt = os.urandom(16)
    #PBKDF2 derivation function
    kdf = PBKDF2HMAC(
    algorithm = hashes.SHA256(),
    length = 64,
    salt = salt,
    iterations = 100000,
    backend = backend
    )
    #password
    password = b"Some password"
    #Encrypt password with PBKDF2
    key = kdf.derive(password)
    #Return key and salt
    return key,salt

def encrypt():
    plainfile = open("file",'rb')
    key, salt = passwd()
    with open('salt','wb') as saltfile:
        saltfile.write(salt)
    keyChaCha20 = key[0:32]
    keySha256 = key[32:]
    cipher = ChaCha20.new(key=keyChaCha20)
    msg = cipher.nonce + cipher.encrypt(plainfile.read())
    h = HMAC.new(keySha256,msg,digestmod=SHA256)
    hex = h.hexdigest()
    with open("ChaCha20",'wb') as chachafile:
        chachafile.write(msg)
    with open("Sha256","w") as shafile:
        shafile.write(hex)

encrypt()

In [5]:
import os

from Crypto.Cipher import Salsa20
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

def passwd():
    backend = default_backend()
    saltfile = open("salt", 'rb')
    salt = saltfile.read()
    #PBKDF2 derivation function
    kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=64,
    salt=salt,
    iterations=100000,
    backend=backend
    )
    #password
    password = b"Some password"
    #Encrypt password with PBKDF2
    key = kdf.derive(password)
    #Get keys for ChaCha20 and Sha256
    secretChaCha20 = key[0:32]
    secretSha256 = key[32:]
    #Return the secrets
    return secretChaCha20, secretSha256

def decrypt():
    secretChaCha20, secretSha256 = passwd()
    with open("ChaCha20", 'rb') as chachafile:
        h = HMAC.new(secretSha256, chachafile.read(), digestmod=SHA256)
    try:
        with open("Sha256",'r') as shafile:
            h.hexverify(shafile.read())
        with open("ChaCha20", 'rb') as chachafile:
            msg_nonce = chachafile.read(8)
            ciphertext = chachafile.read()
        cipher = ChaCha20.new(key=secretChaCha20, nonce=msg_nonce)
        text = cipher.decrypt(ciphertext)
        with open('decrypted','wb') as decryptedfile:
            decryptedfile.write(text)
    except ValueError:
        print("Message has been compromised")
        
decrypt()        